In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### Step 1: Import Options Data from NSE India (Interval: 1 Year)

To begin analyzing options data, we first need to import the data for a 1-year interval from the **NSE India** website or API. This step involves fetching options data for specific stocks (e.g., **RELIANCE**) or indices.



In [ ]:
import requests
import os
from datetime import datetime, timedelta
from tqdm import tqdm

def download_bhavcopy(date, save_dir="bhavcopies"):
    # Create the directory to save files
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Format the URL for the given date
    url = f"https://archives.nseindia.com/content/historical/DERIVATIVES/{date.year}/{date.strftime('%b').upper()}/fo{date.strftime('%d%b%Y').upper()}bhav.csv.zip"

    # File name and path
    file_name = f"fo{date.strftime('%d%b%Y').upper()}bhav.csv.zip"
    file_path = os.path.join(save_dir, file_name)

    # Set custom headers to mimic a browser
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Connection": "keep-alive",
        "Referer": "https://www.nseindia.com/",
    }

    # Download the file
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(file_path, "wb") as f:
            f.write(response.content)
        return f"Downloaded: {file_name}"
    else:
        return f"Failed to download for {date.strftime('%Y-%m-%d')}: {response.status_code}, {response.reason}"

# Specify the date range
start_date = datetime(2024, 1, 2)  # Change to your desired start date
end_date = datetime(2025, 1, 17)   # Change to your desired end date

# Progress bar with tqdm
date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]
for date in tqdm(date_range, desc="Downloading Bhavcopies"):
    result = download_bhavcopy(date)
    print(result)

### Unzip the 1 year interval files and save them in a directory

In [5]:
import os
import zipfile
import pandas as pd
from tqdm import tqdm

def unzip_and_convert_to_csv(directory, output_directory):
    """
    Unzip all ZIP files in a directory, extract CSV files, and save them in the output directory.

    Parameters:
    - directory (str): Path to the directory containing ZIP files.
    - output_directory (str): Path where extracted CSV files will be saved.

    Returns:
    - None
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    zip_files = [f for f in os.listdir(directory) if zipfile.is_zipfile(os.path.join(directory, f))]

    for zip_file in tqdm(zip_files, desc="Processing ZIP files", unit="zip"):
        zip_file_path = os.path.join(directory, zip_file)

        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # Extract all files in the ZIP
            for extracted_file in zip_ref.namelist():
                if extracted_file.endswith('.csv'):  # Ensure we're only dealing with CSVs
                    # Read the CSV content from the ZIP file
                    with zip_ref.open(extracted_file) as f:
                        df = pd.read_csv(f)
                        # Save the CSV file to the output directory
                        output_file_path = os.path.join(output_directory, extracted_file)
                        df.to_csv(output_file_path, index=False)

    print(f"All ZIP files processed and CSVs saved to {output_directory}")

# Example usage
directory_path = '/Users/nihaalnadaf/Documents/Derivative Pricing/bhavcopies'  # Replace with your directory containing ZIP files
output_directory = '/Users/nihaalnadaf/Downloads/op_csv'  # Replace with the directory to save CSVs
unzip_and_convert_to_csv(directory_path, output_directory)

Processing ZIP files: 100%|████████████████| 3592/3592 [09:40<00:00,  6.19zip/s]

All ZIP files processed and CSVs saved to /Users/nihaalnadaf/Downloads/op_csv


### Merge all csv files

In [101]:
import os
import pandas as pd
from tqdm import tqdm

def merge_csv_files(directory, output_csv, chunk_size=100000):
    """
    Merge all CSV files in the specified directory into a single CSV file, processing them in chunks.
    
    Parameters:
    - directory (str): Path to the directory containing the CSV files.
    - output_csv (str): Path to save the merged CSV file.
    - chunk_size (int): Number of rows to read at a time from each CSV file.

    Returns:
    - None
    """
    # List all CSV files in the directory
    csv_files = [file_name for file_name in os.listdir(directory) if file_name.endswith('.csv')]

    # Open the output CSV file in write mode
    with open(output_csv, 'w', newline='', encoding='utf-8') as output_file:
        # Initialize a flag to write the header only once
        header_written = False

        # Loop through all CSV files with tqdm progress bar
        for file_name in tqdm(csv_files, desc="Processing CSV files", unit="file"):
            file_path = os.path.join(directory, file_name)
            
            # Read the CSV file in chunks
            for chunk in pd.read_csv(file_path, chunksize=chunk_size):
                # Write the header only for the first file
                if not header_written:
                    chunk.to_csv(output_file, index=False, header=True)
                    header_written = True
                else:
                    chunk.to_csv(output_file, index=False, header=False)
                    
    print(f"All CSV files merged and saved to {output_csv}")

# Example usage
directory_path = '/Users/nihaalnadaf/Downloads/op_csv' # Replace with the directory path
output_file = '/Users/nihaalnadaf/Downloads/equity/opt_data.csv'  # Replace with the desired output file path

merge_csv_files(directory_path, output_file)

Processing CSV files: 100%|█████████████████████| 7/7 [00:03<00:00,  1.93file/s]

All CSV files merged and saved to /Users/nihaalnadaf/Downloads/equity/opt_data.csv


### Download equity stock price data from nse website

### Concat Equity stock price data and options data on basis of date

In [298]:
df_stk_data=pd.read_csv('/Users/nihaalnadaf/Downloads/equity/stk_data.csv')
df_opt_data=pd.read_csv('/Users/nihaalnadaf/Downloads/equity/opt_data.csv')

In [299]:
df_stk_data.head(5)

,Unnamed: 0,Date,STOCK_PRICE,STOCK_VOLUME,vwap
0,0,31-Dec-2010,1058.70,2279759,1060.46
1,1,30-Dec-2010,1051.20,3328572,1051.30
2,2,29-Dec-2010,1048.60,2556591,1050.21
3,3,28-Dec-2010,1047.90,2024641,1049.90
4,4,27-Dec-2010,1055.25,1321679,1060.51


In [302]:
df_opt_data.head(5)

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,Unnamed: 15
0,OPTSTK,RELIANCE,29-Dec-2011,520.0,CE,0.0,0.0,0.0,301.95,292.75,0.0,0.0,0.0,0.0,07-DEC-2011,NaN
1,OPTSTK,RELIANCE,29-Dec-2011,540.0,CE,0.0,0.0,0.0,283.40,272.85,0.0,0.0,0.0,0.0,07-DEC-2011,NaN
2,OPTSTK,RELIANCE,29-Dec-2011,560.0,CE,0.0,0.0,0.0,265.20,252.95,0.0,0.0,0.0,0.0,07-DEC-2011,NaN
3,OPTSTK,RELIANCE,29-Dec-2011,580.0,CE,0.0,0.0,0.0,247.35,233.10,0.0,0.0,0.0,0.0,07-DEC-2011,NaN
4,OPTSTK,RELIANCE,29-Dec-2011,600.0,CE,0.0,0.0,0.0,230.00,213.20,0.0,0.0,0.0,0.0,07-DEC-2011,NaN


In [304]:
df_opt_data['Date'] = pd.to_datetime(df_opt_data['TIMESTAMP']).dt.date

/var/folders/qn/kq8sg40148x2sxk_3k418fmh0000gn/T/ipykernel_91749/105534337.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_opt_data['Date'] = pd.to_datetime(df_opt_data['TIMESTAMP']).dt.date


In [306]:
df_stk_data['Date']=pd.to_datetime(df_stk_data['Date ']).dt.date

In [308]:
def fix_date_format(date):
    try:
        # Attempt to convert with the original format
        return pd.to_datetime(date, format='%d-%b-%y', errors='raise')
    except ValueError:
        # If conversion fails, fix the date by adding "20" to the year
        if len(date) == 8:  # e.g., '28-Jun-12'
            date = date[:-2] + '20' + date[-2:]  # e.g., '28-Jun-2012'
        return pd.to_datetime(date, format='%d-%b-%Y', errors='raise')

# Iterate through the rows and fix the date if necessary
for idx, date in df_opt_data['EXPIRY_DT'].items():
    try:
        # Attempt to convert the date
        df_opt_data.at[idx, 'EXPIRY_DT'] = fix_date_format(date)
    except ValueError:
        print(f"Error with row {idx}: {date}. Could not convert.")
        # Continue to next row without raising an error

In [309]:
df_opt_data['exp_date'] = pd.to_datetime(df_opt_data['EXPIRY_DT']).dt.date

In [312]:
df=pd.merge(df_stk_data,df_opt_data, on='Date', how='inner')

In [314]:
df.head(5)

,Unnamed: 0,Date,STOCK_PRICE,STOCK_VOLUME,vwap,Date,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,...,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,Unnamed: 15,exp_date
0,1,30-Dec-2010,1051.2,3328572,1051.3,2010-12-30,OPTSTK,RELIANCE,2010-12-30 00:00:00,500.0,...,0.0,516.80,0.0,0.0,0.0,0.0,0.0,30-DEC-2010,NaN,2010-12-30
1,1,30-Dec-2010,1051.2,3328572,1051.3,2010-12-30,OPTSTK,RELIANCE,2010-12-30 00:00:00,550.0,...,0.0,467.70,0.0,0.0,0.0,0.0,0.0,30-DEC-2010,NaN,2010-12-30
2,1,30-Dec-2010,1051.2,3328572,1051.3,2010-12-30,OPTSTK,RELIANCE,2010-12-30 00:00:00,580.0,...,0.0,418.00,0.0,0.0,0.0,0.0,0.0,30-DEC-2010,NaN,2010-12-30
3,1,30-Dec-2010,1051.2,3328572,1051.3,2010-12-30,OPTSTK,RELIANCE,2010-12-30 00:00:00,600.0,...,0.0,398.35,0.0,0.0,0.0,0.0,0.0,30-DEC-2010,NaN,2010-12-30
4,1,30-Dec-2010,1051.2,3328572,1051.3,2010-12-30,OPTSTK,RELIANCE,2010-12-30 00:00:00,620.0,...,0.0,378.75,0.0,0.0,0.0,0.0,0.0,30-DEC-2010,NaN,2010-12-30


In [316]:
df.count()

Unnamed: 0      893539
Date            893539
STOCK_PRICE     893539
STOCK_VOLUME    893539
vwap            893539
Date            893539
INSTRUMENT      893539
SYMBOL          893539
EXPIRY_DT       893539
STRIKE_PR       893539
OPTION_TYP      893539
OPEN            893539
HIGH            893539
LOW             893539
CLOSE           893539
SETTLE_PR       893539
CONTRACTS       893539
VAL_INLAKH      893539
OPEN_INT        893539
CHG_IN_OI       893539
TIMESTAMP       893539
Unnamed: 15          0
exp_date        893539
dtype: int64

In [318]:
df.columns

Index(['Unnamed: 0', 'Date ', 'STOCK_PRICE', 'STOCK_VOLUME', 'vwap ', 'Date',
       'INSTRUMENT', 'SYMBOL', 'EXPIRY_DT', 'STRIKE_PR', 'OPTION_TYP', 'OPEN',
       'HIGH', 'LOW', 'CLOSE', 'SETTLE_PR', 'CONTRACTS', 'VAL_INLAKH',
       'OPEN_INT', 'CHG_IN_OI', 'TIMESTAMP', 'Unnamed: 15', 'exp_date'],
      dtype='object')

In [320]:
df=df.drop(columns=['TIMESTAMP','VAL_INLAKH','EXPIRY_DT','SYMBOL','INSTRUMENT','Date ','Unnamed: 0','Unnamed: 15'])

In [322]:
df.head(5
)

,STOCK_PRICE,STOCK_VOLUME,vwap,Date,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,OPEN_INT,CHG_IN_OI,exp_date
0,1051.2,3328572,1051.3,2010-12-30,500.0,CA,0.0,0.0,0.0,516.80,0.0,0.0,0.0,0.0,2010-12-30
1,1051.2,3328572,1051.3,2010-12-30,550.0,CA,0.0,0.0,0.0,467.70,0.0,0.0,0.0,0.0,2010-12-30
2,1051.2,3328572,1051.3,2010-12-30,580.0,CA,0.0,0.0,0.0,418.00,0.0,0.0,0.0,0.0,2010-12-30
3,1051.2,3328572,1051.3,2010-12-30,600.0,CA,0.0,0.0,0.0,398.35,0.0,0.0,0.0,0.0,2010-12-30
4,1051.2,3328572,1051.3,2010-12-30,620.0,CA,0.0,0.0,0.0,378.75,0.0,0.0,0.0,0.0,2010-12-30


In [324]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['exp_date'] = pd.to_datetime(df['exp_date'], errors='coerce')

# Calculate time to expiry in years and add it as a new column
df['time_to_expiry_years'] = (df['exp_date'] - df['Date']).dt.days / 365

In [326]:
df.head(2)

,STOCK_PRICE,STOCK_VOLUME,vwap,Date,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,OPEN_INT,CHG_IN_OI,exp_date,time_to_expiry_years
0,1051.2,3328572,1051.3,2010-12-30,500.0,CA,0.0,0.0,0.0,516.8,0.0,0.0,0.0,0.0,2010-12-30,0.0
1,1051.2,3328572,1051.3,2010-12-30,550.0,CA,0.0,0.0,0.0,467.7,0.0,0.0,0.0,0.0,2010-12-30,0.0


In [338]:
df.isnull().sum()

STOCK_PRICE             0
STOCK_VOLUME            0
vwap                    0
Date                    0
STRIKE_PR               0
OPTION_TYP              0
OPEN                    0
HIGH                    0
LOW                     0
CLOSE                   0
SETTLE_PR               0
CONTRACTS               0
OPEN_INT                0
CHG_IN_OI               0
exp_date                0
time_to_expiry_years    0
dtype: int64

In [ ]:
import numpy as np
import pandas as pd
from py_vollib.black_scholes import black_scholes
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks import analytical

# Assuming volatility and risk-free rate are provided
volatility = 0.17  # 20% annual volatility
interest_rate = 0.065  # 6.5% annual risk-free rate

# Function to calculate the Greeks for each option
def calculate_greeks(row):
    S = row['STOCK_PRICE']  # Stock price
    K = row['STRIKE_PR']  # Strike price
    T = row['time_to_expiry_years']  # Time to expiry in years
    r = interest_rate  # Risk-free interest rate
    flag = 'c' if row['OPTION_TYP'] in ['CA', 'CE'] else 'p'  # Option type (call or put)
    
    # Calculate Greeks using Black-Scholes model
    delta = analytical.delta(flag, S, K, T, r, volatility)
    gamma = analytical.gamma(flag, S, K, T, r, volatility)
    vega = analytical.vega(flag, S, K, T, r, volatility)
    theta = analytical.theta(flag, S, K, T, r, volatility)
    rho = analytical.rho(flag, S, K, T, r, volatility)  # Rho calculation

    return pd.Series({
        'Delta': delta,
        'Gamma': gamma,
        'Vega': vega,
        'Theta': theta,
        'Rho': rho  # Add Rho to the returned Series
    })

# Apply the function to each row of the DataFrame
greeks_df = df.apply(calculate_greeks, axis=1)

# Join the calculated Greeks to the original DataFrame
df = pd.concat([df, greeks_df], axis=1)

# Print the updated DataFrame
print(df)

In [346]:
df.fillna(0, inplace=True)

In [348]:
df.head(1)

,STOCK_PRICE,STOCK_VOLUME,vwap,Date,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,...,CONTRACTS,OPEN_INT,CHG_IN_OI,exp_date,time_to_expiry_years,Delta,Gamma,Vega,Theta,Rho
0,1051.2,3328572,1051.3,2010-12-30,500.0,CA,0.0,0.0,0.0,516.8,...,0.0,0.0,0.0,2010-12-30,0.0,1.0,0.0,0.0,0.0,0.0


In [350]:
df.to_csv('/Users/nihaalnadaf/Downloads/equity/DF.csv')

### Split the data as per option type (call and put)

In [352]:
df['OPTION_TYP'].unique()

array(['CA', 'PA', 'CE', 'PE'], dtype=object)

In [354]:
df_call = df[df['OPTION_TYP'] == 'CE']

In [356]:
df_call.count()

STOCK_PRICE             419319
STOCK_VOLUME            419319
vwap                    419319
Date                    419319
STRIKE_PR               419319
OPTION_TYP              419319
OPEN                    419319
HIGH                    419319
LOW                     419319
CLOSE                   419319
SETTLE_PR               419319
CONTRACTS               419319
OPEN_INT                419319
CHG_IN_OI               419319
exp_date                419319
time_to_expiry_years    419319
Delta                   419319
Gamma                   419319
Vega                    419319
Theta                   419319
Rho                     419319
dtype: int64

In [358]:
df_put = df[df['OPTION_TYP'] == 'PE']

In [360]:
df_put.count()

STOCK_PRICE             419314
STOCK_VOLUME            419314
vwap                    419314
Date                    419314
STRIKE_PR               419314
OPTION_TYP              419314
OPEN                    419314
HIGH                    419314
LOW                     419314
CLOSE                   419314
SETTLE_PR               419314
CONTRACTS               419314
OPEN_INT                419314
CHG_IN_OI               419314
exp_date                419314
time_to_expiry_years    419314
Delta                   419314
Gamma                   419314
Vega                    419314
Theta                   419314
Rho                     419314
dtype: int64

In [362]:
df_put.to_csv('/Users/nihaalnadaf/Downloads/equity/DF_put.csv')
df_call.to_csv('/Users/nihaalnadaf/Downloads/equity/DF_call.csv')